In [3]:
import google.generativeai as genai
import requests
import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import os

# Set up Google API key
#genai.configure(api_key='YOUR_GOOGLE_API_KEY')
api_key = os.getenv("SECRET_KEY")
genai.configure(api_key=api_key)

In [4]:
# Function to get current location
def get_current_location(zipcode):
    geolocator = Nominatim(user_agent="restaurant_finder")
    #location = geolocator.geocode("Longmont, CO")  # Replace with actual current location method
    #location = geolocator.geocode(zipcode)  # Replace with actual current location method
    print("Finding locations within 1 mile of " + zipcode + ", USA")
    location = geolocator.geocode(zipcode + ", USA")

    return location.latitude, location.longitude

# Function to search for locations using Google Places API
def search_locations(lat, lon, what_to_find, radius=1609):  # 1609 meters = 1 mile
    url = "https://places.googleapis.com/v1/places:searchText"
    payload = {
        "textQuery": what_to_find,
        "location_bias": {
            "circle": {
                "center": {"latitude": lat, "longitude": lon},
                "radius": radius
            }
        },
        "maxResultCount": 10
    }
    #print("search_locations: Payload=")
    #print(payload)
#        'X-Goog-Api-Key': 'YOUR_GOOGLE_API_KEY',
    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': api_key,
        'X-Goog-FieldMask': 'places.id,places.displayName,places.formattedAddress,places.location,places.generativeSummary'
    }
    #print("search_locations: Headers=")
    #print(headers)
    response = requests.post(url, headers=headers, json=payload)
    #print("search_locations: response.json()")
    #print(response.json())
    return response.json()

# Function to generate summary using Google's generative AI
def generate_summary(info):
    model = genai.GenerativeModel('gemini-pro')
    prompt = f"Summarize this location in 2-3 sentences: {info}"
    response = model.generate_content(prompt)
    return response.text

# Function to display locations on a map
def display_locations_on_map(current_lat, current_lon, locations):
    #print("Number of items in locations list:", len(locations))
    #print("First item in locations list:", locations[0])
    
    m = folium.Map(location=[current_lat, current_lon], zoom_start=14)
    folium.Marker([current_lat, current_lon], popup="You are here", icon=folium.Icon(color='red')).add_to(m)
    
    for place in locations.get('places', []):
        lat = place['location']['latitude']
        lon = place['location']['longitude']
        name = place['displayName']['text']
        address = place['formattedAddress']
        summary = generate_summary(f"{name} - {address}")
        
        folium.Marker(
            [lat, lon],
            popup=f"{name}<br>{address}<br><br>{summary}"
        ).add_to(m)
    
    return m


In [5]:

# Main execution
print("Please input the zip code of where you want to find something?")
zipcode = input()
print("Please input what you want to search for?")
what_to_find = input()

current_lat, current_lon = get_current_location(zipcode)
locations = search_locations(current_lat, current_lon, what_to_find)
map_with_locations = display_locations_on_map(current_lat, current_lon, locations)

# Display the map
map_with_locations


Please input the zip code of where you want to find something?
Please input what you want to search for?
Finding locations within 1 mile of 80027, USA
